In [1]:
import numpy as np


# Define Rastrigin Function


In [2]:
def rastrigin(X):
    if isinstance(X[0],(int,float)):
        X = [[X[i]] for i in range(len(X))]
    
    val = []
    for xi in X:
        fx = 10 * len(xi) + sum(np.array(xi) ** 2 - 10 * np.cos(2 * np.pi * np.array(xi)))
        val.append(fx)
    return np.array(val)

# Initiate population

In [3]:
def generate_particle(num_variables,swarm_size,x_min,x_max):
    X = []
    for i in range(swarm_size):
        Xi = []
        for j in range(num_variables):
            Xi.append(np.random.uniform(x_min,x_max)) #testing with integers so easier to crosscheck
        X.append(Xi)

    velocity = np.zeros_like(X)

    return X , velocity

# Particle Swarm Optimisation

In [4]:
def pso1(num_iterations,swarm_size,num_variables,x_min,x_max,alpha,beta,gamma,epsilon):
    swarm,velocity = generate_particle(num_variables,swarm_size,x_min,x_max)

    for i in range(num_iterations):

        swarm2 = swarm + epsilon * velocity

        dicx= {tuple(swarm[i]):rastrigin(swarm)[i] for i in range(swarm_size)}
        dicy= {tuple(swarm2[i]):rastrigin(swarm2)[i] for i in range(swarm_size)}

        if i == 0:
            local_best = swarm
            global_best = list(min(dicx, key=lambda k: dicx[k]))

        else:
             for j in range(swarm_size):

                 key1,value1 = list(dicx.items())[j]
                 key2,value2 = list(dicy.items())[j]

                 if value2 < value1:
                     local_best[j] = list(key2)
                 else:
                     local_best[j] = list(key1)

                 if np.argmin(dicy)<np.argmin(dicx):
                     global_best = list(min(dicy, key=lambda k: dicy[k]))
                 else:
                     global_best = list(min(dicx, key=lambda k: dicx[k]))                

        #update velocity
        velocity = alpha * velocity +  np.random.uniform(0,beta) * (np.array(local_best) - np.array(swarm)) + np.random.uniform(0,gamma) * (np.full_like(swarm,global_best) - np.array(swarm))    #resize the global best

    return dicy

In [5]:

def index_finder(arr):
    mini = 0
    for i in range(len(arr)):
        if rastrigin(arr)[i] < rastrigin(arr)[mini]:
            mini = i
    return mini 

In [15]:
def pso(num_iterations,swarm_size,num_variables,x_min,x_max,W,C1,C2,E):
    swarm,velocity = generate_particle(num_variables,swarm_size,x_min,x_max)
    local_best = np.copy(swarm)
    global_best = np.full_like(local_best,local_best[index_finder(swarm)])
    
    for z in range(num_iterations):
        swarm2 = swarm + (E * velocity)
        swarm2 = np.clip(swarm2,x_min,x_max)
        
        ras_swarm = np.array(rastrigin(swarm))
        ras_swarm2 = np.array(rastrigin(swarm2))
        
        for i in range(swarm_size):
            if ras_swarm2[i] < ras_swarm[i]:
                local_best[i] = swarm2[i]
                continue
            if min(ras_swarm2) < min(ras_swarm):
                global_best = np.full_like(local_best,local_best[index_finder(swarm2)])
                print(global_best[0])
                continue
            
        velocity = ((W*(W-.4)*velocity*z)/num_iterations) + C1 * np.random.uniform(0,1) * (local_best - swarm) + C2 * np.random.uniform(0,1) * (global_best - swarm)
    
    return global_best[0]

In [16]:
print(pso(100,50,5,-5.12,+5.12,.9,.2,.7,.2))

[ 0.34155426 -1.89896636 -2.40557765  0.26526674 -1.86793206]
[ 0.34155426 -1.89896636 -2.40557765  0.26526674 -1.86793206]
[ 0.34155426 -1.89896636 -2.40557765  0.26526674 -1.86793206]
[ 0.34155426 -1.89896636 -2.40557765  0.26526674 -1.86793206]
[ 0.34155426 -1.89896636 -2.40557765  0.26526674 -1.86793206]
[ 0.34155426 -1.89896636 -2.40557765  0.26526674 -1.86793206]
[ 0.34155426 -1.89896636 -2.40557765  0.26526674 -1.86793206]
[ 0.34155426 -1.89896636 -2.40557765  0.26526674 -1.86793206]
[ 0.23525935 -1.85176851 -2.16718184  0.11162446 -1.78327786]
[ 0.23525935 -1.85176851 -2.16718184  0.11162446 -1.78327786]
[ 0.23525935 -1.85176851 -2.16718184  0.11162446 -1.78327786]
[ 0.23525935 -1.85176851 -2.16718184  0.11162446 -1.78327786]
[ 0.23525935 -1.85176851 -2.16718184  0.11162446 -1.78327786]
[ 0.23525935 -1.85176851 -2.16718184  0.11162446 -1.78327786]
[ 0.23525935 -1.85176851 -2.16718184  0.11162446 -1.78327786]
[ 0.23525935 -1.85176851 -2.16718184  0.11162446 -1.78327786]
[ 0.2352